In [38]:
# Settings

# General settings
data_folder = "/home/andrea/Projects/WMO_Volta/data/data_dynamic/discharge/" #"/home/andrea/Desktop/Test/series/" 
start_analysis = "2018-01-01 00:00"
end_analysis = "2020-12-30 00:00"
freq="D"

# Hmc settings
domain = "volta"
io_path = "/home/andrea/Projects/WMO_Volta/test/compare/rainfall_analysis/imerg/beta1_low/"
hmc_output = "/home/andrea/Projects/WMO_Volta/test/compare/hmc.hydrograph_full_no_ia.txt"
hmc_static_gridded = "/home/andrea/Projects/WMO_Volta/data/data_static/volta_imerg/gridded/"
hmc_static_point = "/home/andrea/Projects/WMO_Volta/data/data_static/volta_imerg/point/"

# Data
et_series = io_path + "average_ETCum.txt"
et_pot_series = io_path + "average_ETPotCum.txt" 
rainfall_series = "/home/andrea/Projects/WMO_Volta/test/compare/rainfall_analysis/imerg/average_rainfall_mm_h.txt"

In [49]:
# Imports
%matplotlib inline

from ipywidgets import interactive,HBox
import pandas as pd
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
from pysheds.grid import Grid

import ipywidgets as widgets
from IPython.display import display, HTML
import os, glob
import datetime as dt
import matplotlib.lines as mlines

# Define custom functions
def create_df(choices):
    df = pd.DataFrame(index=pd.date_range(start_time,end_time,freq=freq), columns=choices)
    for name in choices:
        series = pd.read_csv(os.path.join(data_folder, name + ".csv"), index_col=0, header=0, parse_dates=True)
        df[name] = series.reindex(pd.date_range(start_time,end_time,freq=freq), method=None)
    return df

def multiplot(widg):
    choices = widg['new']
    df = create_df(choices)
    data = df.loc[:, choices] if choices else df
    output.clear_output(wait=True)
    with output:
        ax = data.plot(figsize=(10,7))
        plt.show()
        
def combinedplot(widg):
    choices = widg['new']
    data_etPot = series["discharge_mod2"].loc[:, choices] if choices else series
    data_et = series["ET"].loc[:, choices] if choices else series
    data_rain = series["rain"].loc[:, choices] if choices else series
    data_qMod = series["discharge_mod"].loc[:, choices] if choices else series
    data_qObs = series["discharge_obs"].loc[:, choices] if choices else series
    output2.clear_output(wait=True)
    with output2:
        ax = data_rain.plot(figsize=(15,7), color='c') 
        plt.ylim(bottom=0)
        plt.ylabel("mm of rain")
        axx = ax.twinx()
        #ax1 = data_et.plot(figsize=(15,7), color='g')
        ax4 = data_etPot.plot(figsize=(15,7), color='k')
        ax2 = data_qMod.plot(figsize=(15,7), color='b')
        ax3 = data_qObs.plot(figsize=(15,7), color='r', style='.')
        plt.ylim(bottom=0)
        plt.ylabel("mm")
        cyan_line = mlines.Line2D([], [], color='cyan', label='rain')
        green_line = mlines.Line2D([], [], color='green', label='et')
        yellow_line = mlines.Line2D([], [], color='black', label='dis_mod_old')
        blue_line = mlines.Line2D([], [], color='blue', label='mod_dis')
        red_line = mlines.Line2D([], [], color='red', linestyle='', marker='.', label='obs_dis')
        
        plt.legend(handles=[cyan_line, green_line, yellow_line, blue_line, red_line])
        plt.show()

def read_discharge_hmc(file='', col_names=None):
    custom_date_parser = lambda x: dt.datetime.strptime(x, "%Y%m%d%H%M")
    hmc_discharge_df = pd.read_csv(file, header=None, delimiter=r"\s+", parse_dates=[0], index_col=[0], date_parser=custom_date_parser)
    hmc_discharge_df.columns = col_names
    return hmc_discharge_df

In [50]:
# Read static inputs

# Sections
sections = tabular = pd.read_csv(os.path.join(hmc_static_point, domain + ".info_section.txt"), sep="\s+", header=None)
rHMC, cHMC, basin_name, section_name = tabular.values[:,0], tabular.values[:,1], tabular.values[:,2], tabular.values[:,3]

# Pointers
grid = Grid.from_ascii(os.path.join(hmc_static_gridded, domain + ".pnt.txt"))
pnt = grid.read_ascii(os.path.join(hmc_static_gridded, domain + ".pnt.txt"), dtype=np.int8)
areacell = grid.read_ascii(os.path.join(hmc_static_gridded, domain + ".areacell.txt"))

In [51]:
# Compute basin cell area
basin_area = pd.DataFrame(index=section_name, columns=["Area (m2)"])
dirmap_HMC = (8, 9, 6, 3, 2, 1, 4, 7)

for ix, iy, basin, name in zip(cHMC, rHMC, basin_name, section_name):
        basin = grid.catchment(fdir=pnt, x=ix-1, y=iy-1, dirmap=dirmap_HMC, xytype='index')
        mask = np.where(basin>0, 1, np.nan)
        basin_area.loc[name, "Area (m2)"] = np.nansum(mask*areacell).astype("float32")

In [59]:
# Read hmc output
mod_out = read_discharge_hmc(hmc_output, section_name)
mod_out_old = read_discharge_hmc("/home/andrea/Projects/WMO_Volta/test/compare/hmc.hydrograph_full_low.txt", section_name)

In [60]:
# Set timing
start_time = dt.datetime.strptime(start_analysis, "%Y-%m-%d %H:%M")
end_time = dt.datetime.strptime(end_analysis, "%Y-%m-%d %H:%M")

In [61]:
# Read datasets
series = {}

series["ET"] = pd.read_csv(et_series, index_col=0, header=0, parse_dates=True)[start_time:end_time]
series["ETPot"] = pd.read_csv(et_pot_series, index_col=0, header=0, parse_dates=True)[start_time:end_time]
series["rain"] = pd.read_csv(rainfall_series, index_col=0, header=0, parse_dates=True)[start_time:end_time]
dis_out = mod_out[start_time:end_time]
dis_out_mod = mod_out_old[start_time:end_time]

missing_data = [i for i in section_name if not os.path.isfile(os.path.join(data_folder, i + ".csv"))]
display("WARNING! Data for sections " + ", ".join(missing_data) + " are missing!")
dis_in = create_df([i for i in section_name if i not in missing_data])

'WARNING! Data for sections Arly_doudouro, Atchangbade, Bagre_aval_au_pont, Bassar, Bayé, Borgou, Brouffou, Ekumdipe, Gbanlou-bineda_Kopingue, Gorée, Gougoulo_Doropo, Kara-kpessidè, Koulda_Pouon, Koumongou, Kounou, LeryNord, Naboulgou, Nangodi, Nano, Nebou, Niaogho, Nnaboupi, Noberé, Noumbiel, Pletou, Porga, Pouda, Prang, Rte-SokodeBassar, Samboali, Sanogo, Tagou, Tampelga, Titira, Tourouba, Volta_Vonkoro, Wiasi, Yilou are missing!'

In [62]:
# Convert all series to mm/day
series["rain"] = series["rain"].resample("D").sum()
series["ET"] = series["ET"].resample("D").sum()
series["ETPot"] = series["ETPot"].resample("D").sum()
series["discharge_mod2"] = pd.DataFrame(index = series["rain"].index, columns = dis_out.columns)
series["discharge_mod"] = pd.DataFrame(index = series["rain"].index, columns = dis_out.columns)
series["discharge_obs"] = pd.DataFrame(index = series["rain"].index, columns = dis_in.columns)
series["availability"] = pd.DataFrame(index = series["rain"].index, columns = dis_in.columns)

for name in dis_out.columns:
    series["discharge_mod"][name] = (dis_out[name] * (1000*3600)/float(basin_area.loc[name, "Area (m2)"])).resample("D").sum()
    series["discharge_mod2"][name] = (dis_out_mod[name] * (1000*3600)/float(basin_area.loc[name, "Area (m2)"])).resample("D").sum()
    
for name in series["discharge_obs"].columns:
    series["discharge_obs"][name] = (dis_in[name] * (1000*3600*24)/float(basin_area.loc[name, "Area (m2)"]))
    series["availability"][name] = dis_in[name].resample("D").count()

In [63]:
series_annual = {}

for stype in series.keys():
    series_annual[stype] = series[stype].resample("Y").sum()
    index = pd.Index([str(y) + "_" + stype for y in np.unique(series_annual[stype].index.year)])
    series_annual[stype].set_index(index, inplace=True)

In [64]:
out_table = pd.concat([series_annual[i].T for i in series_annual.keys()], axis=1)
out_table = out_table.reindex(sorted(out_table.columns), axis=1)

display(out_table)
et_rate_value = 100*((out_table["2018_ET"]/out_table["2018_rain"]) + (out_table["2019_ET"]/out_table["2019_rain"]) + (out_table["2020_ET"]/out_table["2020_rain"]))/3
et_rate = pd.DataFrame(et_rate_value, index=out_table.index, columns=["rate"])
et_rate.to_csv(io_path + "/et_rate.csv")
out_table.to_csv(io_path + "/summary_balance.csv")

,2018_ET,2018_ETPot,2018_availability,2018_discharge_mod,2018_discharge_mod2,2018_discharge_obs,2018_rain,2019_ET,2019_ETPot,2019_availability,...,2019_discharge_mod2,2019_discharge_obs,2019_rain,2020_ET,2020_ETPot,2020_availability,2020_discharge_mod,2020_discharge_mod2,2020_discharge_obs,2020_rain
Arly_doudouro,764.522405,1837.791784,NaN,42.735558,35.975788,NaN,790.921938,851.957632,1837.391717,NaN,...,58.009212,NaN,970.648474,908.556130,1853.311914,NaN,147.558783,52.788296,NaN,956.784302
Atchangbade,1112.272651,1212.569368,NaN,195.061852,69.858996,NaN,1160.264104,1076.046955,1213.045944,NaN,...,117.613911,NaN,1226.638043,1147.655122,1278.029498,NaN,412.291636,289.545308,NaN,1469.368808
Badara,970.869916,1758.424343,290.0,241.816104,83.736469,79.559732,1242.660414,1202.349739,1782.794119,299.0,...,102.055644,49.561959,1314.401025,1165.004842,1788.667681,312.0,206.107288,45.294182,66.444979,1082.414506
Bagre_aval_au_pont,819.269263,1934.326171,NaN,62.358065,44.039171,NaN,899.876780,809.326980,1954.806552,NaN,...,48.262376,NaN,866.015421,847.546175,1980.145757,NaN,189.090984,63.202770,NaN,945.909912
Bamboi,876.756117,1755.068068,365.0,95.897862,48.087188,73.350016,984.128883,925.791771,1769.056743,365.0,...,97.628449,89.788584,1065.681059,921.726342,1785.210632,0.0,100.340526,55.372588,0.000000,892.396594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yilou,826.691752,1986.970146,NaN,108.563371,53.567023,NaN,948.616621,774.010823,2016.471412,NaN,...,38.646073,NaN,765.908095,751.150219,2021.773637,NaN,102.054818,46.086386,NaN,817.367669
Ziou,863.720376,1821.827583,359.0,140.761805,47.630269,56.807634,953.586745,926.950815,1830.015053,363.0,...,80.628357,44.856950,1051.821795,969.529803,1860.589221,274.0,254.258480,72.712729,49.546870,1037.767736
Gbanlou-bineda_Kopingue,NaN,NaN,NaN,74.837776,74.181917,NaN,NaN,NaN,NaN,NaN,...,160.552791,NaN,NaN,NaN,NaN,NaN,37.326559,76.468354,NaN,NaN
Kara-kpessidè,NaN,NaN,NaN,320.136203,113.454905,NaN,NaN,NaN,NaN,NaN,...,151.955520,NaN,NaN,NaN,NaN,NaN,604.100300,342.534758,NaN,NaN


In [65]:
# Generate list
selector2 = widgets.Dropdown(
options=series_annual["ET"].T.index,
value=series_annual["ET"].T.index[0])

output2 = widgets.Output()

# Set layout and display
form_item_layout = widgets.Layout(
    display='flex',
    justify_content='space-between'
)
display(widgets.VBox([selector2, output2], layout=form_item_layout))

# Re-generate multiplot
selector2.observe(combinedplot, names='value')